## Demo

In [1]:
# Install necessary libraries (run only if not already installed)
# !pip install ipywidgets scikit-learn matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display
from sklearn.metrics import mean_squared_error

# --- 1. Use your improved dataset with scaling ---
np.random.seed(42)
X = np.linspace(0, 1, 100).reshape(-1, 1)
true_function = lambda x: 3 * np.sin(10 * x) + x**2
y = true_function(X) + np.random.normal(0, 0.5, X.shape)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

# --- 2. Interactive function ---
def update_plot(degree=3, regularization='None', alpha=0.01):
    # Choose model
    if regularization == 'None':
        model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
        title = f"Linear Regression (Degree={degree})"
    elif regularization == 'Ridge':
        model = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=alpha))
        title = f"Ridge Regression (Degree={degree}, alpha={alpha})"
    else:
        model = make_pipeline(PolynomialFeatures(degree), Lasso(alpha=alpha, max_iter=10000))
        title = f"Lasso Regression (Degree={degree}, alpha={alpha})"
    
    model.fit(X_train, y_train)
    
    x_plot = np.linspace(X_scaled.min(), X_scaled.max(), 100).reshape(-1, 1)
    y_pred = model.predict(x_plot)
    
    # Calculate errors (even though we're not displaying them)
    train_mse = mean_squared_error(y_train, model.predict(X_train))
    val_mse = mean_squared_error(y_val, model.predict(X_val))

    # --- 3. Plot only the model fit ---
    plt.figure(figsize=(10, 6))
    
    plt.scatter(X_train, y_train, color='blue', label="Training Data")
    plt.scatter(X_val, y_val, color='red', label="Validation Data")
    plt.plot(x_plot, y_pred, color='black', label="Model Prediction")
    plt.title(title)
    plt.xlabel("X (scaled)")
    plt.ylabel("y (scaled)")
    plt.legend()
    plt.grid(True)
    plt.ylim(-3, 3)
    
    # Add error information as text instead of graph
    plt.annotate(f"Training MSE: {train_mse:.4f}\nValidation MSE: {val_mse:.4f}", 
                 xy=(0.02, 0.96), xycoords='axes fraction',
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

    plt.tight_layout()
    plt.show()

# --- 4. Widgets ---
degree_slider = widgets.IntSlider(value=3, min=1, max=30, step=1, description='Degree:')
alpha_slider = widgets.FloatLogSlider(value=0.01, base=10, min=-4, max=1, step=0.1, description='Alpha:')
reg_select = widgets.Dropdown(options=['None', 'Ridge', 'Lasso'], value='None', description='Reg:')

ui = widgets.VBox([degree_slider, reg_select, alpha_slider])
out = widgets.interactive_output(update_plot, {
    'degree': degree_slider,
    'regularization': reg_select,
    'alpha': alpha_slider
})

# --- 5. Display ---
display(ui, out)

Output()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from IPython.display import display

# --- 1. Generate and scale data ---
np.random.seed(42)
X = np.linspace(0, 1, 100).reshape(-1, 1)
true_function = lambda x: 3 * np.sin(10 * x) + x**2
y = true_function(X) + np.random.normal(0, 0.5, X.shape)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

# --- 2. Interactive error plot function ---
def plot_errors(regularization='None', alpha=0.01):
    degrees = range(1, 30)
    train_errors = []
    val_errors = []

    for degree in degrees:
        if regularization == 'None':
            model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
        elif regularization == 'Ridge':
            model = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=alpha))
        else:
            model = make_pipeline(PolynomialFeatures(degree), Lasso(alpha=alpha, max_iter=10000))

        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        train_errors.append(mean_squared_error(y_train, y_train_pred))
        val_errors.append(mean_squared_error(y_val, y_val_pred))

    # --- 3. Plot ---
    plt.figure(figsize=(10, 6))
    plt.plot(degrees, train_errors, label="Training Error", color='blue', marker='o')
    plt.plot(degrees, val_errors, label="Validation Error", color='red', marker='o')
    plt.yscale('log')
    plt.xlabel("Polynomial Degree")
    plt.ylabel("Mean Squared Error")
    plt.title(f"Bias-Variance Tradeoff with {regularization} Regression (alpha={alpha})")
    plt.legend()
    plt.grid(True)
    plt.show()

# --- 4. Interactive widgets ---
reg_select = widgets.Dropdown(options=['None', 'Ridge', 'Lasso'], value='None', description='Reg:')
alpha_slider = widgets.FloatLogSlider(value=0.01, base=10, min=-4, max=1, step=0.1, description='Alpha:')

ui = widgets.VBox([reg_select, alpha_slider])
out = widgets.interactive_output(plot_errors, {
    'regularization': reg_select,
    'alpha': alpha_slider
})

# --- 5. Display ---
display(ui, out)


Output()